In [1]:
import re
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
import convertTxtToDFFns
from xml2txt import bulkXml2Txt
from pdf2xml import pdf2xml

In [25]:
with open('cuck.txt') as file:
    fullChart = file.readlines()

In [51]:
line = ' BELMO<NT PARK - June 8, 2019 - Race 11'
line = ' One And One Half Miles On The Dirt Track Record: (Secretariat - 2:24.00 - June 9, 1973)'

In [ ]:
def parseFullDay(fullChart):
newRaceInd = [-1] #first index will be 0 after conversion (controls for out of index error later)
newRaceTest = 'Copyright 2020 Equibase Company LLC. All Rights Reserved.'
for i in range(len(fullChart)):
    if re.search(newRaceTest, fullChart[i]) is not None: #check for the expression that signifies end of race
        newRaceInd.append(i)

    dayDF = pd.DataFrame(columns=['trackName','month','day','year','raceNum','breed','distance','surface','segment1','segment2','segment3','segment4','segment5','segments'])

    for i in range(len(newRaceInd) - 1):

        raceDF = parseRace(fullChart[newRaceInd[i] + 1:newRaceInd[i + 1]]) #loop over chunks of the file, each chunk representing a full race
        if type(raceDF) == str:
            continue
        else:
            dayDF = pd.concat([dayDF, raceDF])

    return dayDF

In [ ]:
def parseRace(raceChart):

    cnt = 0

    for line in raceChart:
        if re.search('Cancelled *- *', line) is not None:
            return 'Error - cancelled'
        elif re.search('Track *Record:', line) is not None:
            trackRecordInd = cnt
        elif re.search('Off *at:', line) is not None:
            genInd = cnt + 2
        elif re.search('Last *Raced *Pgm', line) is not None:
            horseInd = [cnt + 1]
        elif re.search('Fractional *Times:', line):
            horseInd.append(cnt)
            endInd = cnt
        cnt += 1
    
    genItems = parseGenInfo(raceChart[:genInd], trackRecordInd=trackRecordInd)

    if genItems == 'Error':
        return 'Error - Quarter Horse'
    
    if len(raceChart) < horseInd[1]:
        jack = 1
    horseItems = parseHorseInfo(raceChart[horseInd[0]:horseInd[1]])
    #endItems = parseEndInfo(raceChart[endInd:])

    genDF = pd.DataFrame(horseItems)
    
    return genDF

In [29]:
newRaceInd = [-1] #first index will be 0 after conversion (controls for out of index error later)
newRaceTest = 'Copyright 2020 Equibase Company LLC. All Rights Reserved.'
for i in range(len(fullChart)):
    if re.search(newRaceTest, fullChart[i]) is not None: #check for the expression that signifies end of race
        newRaceInd.append(i)
for i in range(len(newRaceInd) - 1):
    print(fullChart[newRaceInd[i] + 1:newRaceInd[i + 1]]) #loop over chunks of the file, each chunk representing a full race

[' BELMO<NT PARK - June 8, 2019 - Race 11\n', ' STAKES Belmont S. Grade 1 - Thoroughbred\n', ' FOR THREE YEAR OLDS. By subscription of $600 each, to accompany the nomination, if made on or before January 26, 2019, or\n', ' $6,000, if made on or before April 1, 2019. At any time prior to the closing time of entries, horses may be nominated to The\n', ' BelmontStakes upon payment of a supplementary fee of $50,000 to the New York Racing Association, Inc. $15,000 to pass the\n', ' entry box and $15,000 additional to start. All entrants will be required to pay entry and starting fees; but no fees, supplemental or\n', ' otherwise shall be added to the purse. The purse to be divided $800,000 to the winner, $280,000 to second, $150,000 to third,\n', ' $100,000 to fourth, $60,000 to fifth, $45,000 to sixth, $35,000 to seventh and $30,000 to eighth. Colts and Geldings, 126 lbs.; Fillies,\n', ' 121 lbs. The winning owner will be presented with the August Belmont Memorial Cup to be retained for on

In [26]:
fullChart

[' BELMO<NT PARK - June 8, 2019 - Race 11\n',
 ' STAKES Belmont S. Grade 1 - Thoroughbred\n',
 ' FOR THREE YEAR OLDS. By subscription of $600 each, to accompany the nomination, if made on or before January 26, 2019, or\n',
 ' $6,000, if made on or before April 1, 2019. At any time prior to the closing time of entries, horses may be nominated to The\n',
 ' BelmontStakes upon payment of a supplementary fee of $50,000 to the New York Racing Association, Inc. $15,000 to pass the\n',
 ' entry box and $15,000 additional to start. All entrants will be required to pay entry and starting fees; but no fees, supplemental or\n',
 ' otherwise shall be added to the purse. The purse to be divided $800,000 to the winner, $280,000 to second, $150,000 to third,\n',
 ' $100,000 to fourth, $60,000 to fifth, $45,000 to sixth, $35,000 to seventh and $30,000 to eighth. Colts and Geldings, 126 lbs.; Fillies,\n',
 ' 121 lbs. The winning owner will be presented with the August Belmont Memorial Cup to be retaine

In [36]:
trackLongToShort = {}
trackShortToLong = {}
tracksDF = pd.read_csv('./../excel/tracks_v03.csv', delimiter=',', header=None)
for i in range(tracksDF.shape[0]):
    trackLongToShort[tracksDF.iloc[i,1]] = tracksDF.iloc[i,0]
    trackShortToLong[tracksDF.iloc[i,0]] = tracksDF.iloc[i,1]

In [43]:
monthNameToNumber = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}